In [1]:
import os
import pandas as pd
import numpy as np
from sklearn import clone
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
from ML_util import *
from ML_Analysis_OptimalStimuli import LoadSelectiveData

path = "data/BlueMediumRarePupil_task1-1.csv"
thisData = LoadSelectiveData(path)

Xdata = thisData.take_x().to_numpy()
Ydata = thisData.take_y().to_numpy()

kf = StratifiedKFold(n_splits=10)
rf_model = RandomForestClassifier(random_state=0)  # Random Forest

shape_cm = np.zeros((13,13), dtype=np.float64)
orientation_cm = np.zeros((13,13), dtype=np.float64)
size_cm = np.zeros((13,13), dtype=np.float64)
hue_cm = np.zeros((13,13), dtype=np.float64)
brightness_cm = np.zeros((13,13), dtype=np.float64)
total_cm = np.zeros((13,13), dtype=np.float64)
log_dict = {f"true_{i}":{f"pred_{j}":[] for j in range(13)} for i in range(13)}

thisData.get_data()

,new_x1,new_y1,new_x2,new_y2,new_x3,new_y3,new_x4,new_y4,new_x5,new_y5,...,target_color_b,target_color_g,target_color_r,distractor_color_b,distractor_color_g,distractor_color_r,target_orientation,distractor_orientation,Task Encoding,Similarity Encoding
0,100.409163,15.556349,99.702056,19.091883,138.592929,-7.071068,108.187337,7.778175,112.429978,7.778175e+00,...,204,204,204,102,102,102,0,0,5,2
1,16.263456,41.719300,19.091883,37.476659,17.677670,33.234019,11.313709,38.183766,19.798990,3.394113e+01,...,62,127,101,62,127,101,0,0,2,2
2,100.681641,-71.744039,104.945679,-77.384782,97.863948,-74.583160,78.011532,-60.516121,92.201777,-7.319038e+01,...,101,62,127,82,62,127,0,0,4,1
3,33.000000,-61.000000,63.000000,-66.000000,24.000000,-39.000000,35.000000,-50.000000,34.000000,-5.000000e+01,...,127,62,82,127,62,101,0,0,4,1
4,23.334524,7.778175,29.698485,18.384776,30.405592,16.263456,33.941125,16.970563,31.819805,1.484924e+01,...,62,82,127,82,62,127,0,0,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12526,114.732553,-141.143336,66.884448,-57.623525,59.681508,-46.304618,21.780320,-25.896286,-10.289915,-5.487955e+00,...,102,102,102,153,153,153,0,0,5,1
12527,-7.071068,1.414214,-8.485281,-4.242641,-8.485281,-1.414214,-8.485281,-1.414214,-7.071068,8.440000e-15,...,62,88,127,62,107,127,0,0,4,1
12528,116.672619,58.689863,114.551299,59.396970,114.551299,57.982756,118.086832,60.104076,115.965512,5.656854e+01,...,62,107,127,62,107,127,0,0,1,2
12529,19.415497,20.592194,4.314555,16.473755,-0.588348,7.256297,-1.961161,0.392232,14.120362,-7.844645e-01,...,62,101,127,101,62,127,0,0,4,4


In [20]:
for train, test in kf.split(Xdata, Ydata):
    clf = clone(rf_model)
    clf.fit(Xdata[train], Ydata[train])
    pred_y = clf.predict(Xdata[test])
    cm = confusion_matrix(Ydata[test], pred_y)
    total_cm = np.add(total_cm, cm)
    for n, index in enumerate(test):
        this_pred_y = pred_y[n]
        this_true_y = Ydata[index]
        this_meta = thisData.take_meta(index)
        log_dict[f"true_{this_pred_y}"][f"pred_{this_true_y}"].append(this_meta)      

In [21]:
for i in range(13):
    for j in range(13):
        this_data = log_dict[f"true_{i}"][f"pred_{j}"]
        for meta in this_data:
            if meta['task'] == 'shape':
                shape_cm[i,j] += 1
            elif meta['task'] == 'size':
                size_cm[i,j] += 1
            elif meta['task'] == 'brightness':
                brightness_cm[i,j] += 1
            elif meta['task'] == 'hue':
                hue_cm[i,j] += 1
            elif meta['task'] == 'orientation':
                orientation_cm[i,j] += 1

In [26]:
np.save(os.path.join(os.getcwd(), 'ml-results', 'medium', 'new_task_cm', 'pupil_task1_shape_cm.npy'), shape_cm)
np.save(os.path.join(os.getcwd(), 'ml-results', 'medium', 'new_task_cm', 'pupil_task1_size_cm.npy'), size_cm)
np.save(os.path.join(os.getcwd(), 'ml-results', 'medium', 'new_task_cm', 'pupil_task1_brightness_cm.npy'), brightness_cm)
np.save(os.path.join(os.getcwd(), 'ml-results', 'medium', 'new_task_cm', 'pupil_task1_hue_cm.npy'), hue_cm)
np.save(os.path.join(os.getcwd(), 'ml-results', 'medium', 'new_task_cm', 'pupil_task1_orientation_cm.npy'), orientation_cm)
np.save(os.path.join(os.getcwd(), 'ml-results', 'medium', 'new_task_cm', 'pupil_task1_total_cm.npy'), total_cm)


In [ ]:
visualize_cm(total_cm, 'RF', 'Pupil', iv=True)


In [6]:
set(thisData.take_y().to_list())

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

In [2]:
for train, test in kf.split(Xdata, Ydata):
    clf = clone(rf_model)
    clf.fit(Xdata[train], Ydata[train])
    testX = Xdata[test]
    testY = Ydata[test]
    stack_index, stack_y = stack_ydata_from_same_combinations(testY, 3)
    results = latefusion(clf, testX, stack_index, stack_y)
    total_cm = np.add(total_cm, results["cm_multiply"])
  


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}


100%|██████████| 413/413 [00:00<00:00, 59002.27it/s]

[[26  0  0  1  0  1  3  0  1  0  0  0  0]
 [ 0 24  1  0  4  0  0  0  0  1  0  3  0]
 [ 0  0 26  0  0  0  0  0  0  0  2  3  0]
 [ 2  1  1 18  0  1  0  3  0  0  1  3  1]
 [ 0  3  0  0 28  0  0  0  0  1  0  0  1]
 [ 4  0  0  0  1 22  4  1  0  0  0  0  0]
 [ 3  0  0  0  0  2 26  1  0  0  0  0  0]
 [ 0  1  0  4  0  2  1 24  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0 30  0  0  0  0]
 [ 0  2  0  0  0  0  0  0  0 28  0  1  1]
 [ 0  0  1  0  0  1  0  0  0  0 27  0  0]
 [ 0  0  2  0  0  1  1  0  0  0  0 28  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 33]]


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}


100%|██████████| 413/413 [00:00<00:00, 55160.09it/s]

[[20  0  0  2  0  4  4  0  2  0  0  0  0]
 [ 0 23  0  1  3  1  0  0  0  1  2  0  2]
 [ 0  0 28  0  0  0  0  0  0  0  1  2  0]
 [ 1  0  0 23  0  0  2  4  0  0  0  1  0]
 [ 0  1  1  0 30  1  0  0  0  0  0  0  0]
 [ 4  1  0  0  1 18  6  1  0  0  0  1  0]
 [ 3  0  0  1  0  2 22  3  1  0  0  0  0]
 [ 1  0  0  0  0  2  0 29  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 31  0  0  0  0]
 [ 0  3  0  0  1  0  0  0  0 26  0  0  2]
 [ 0  0  0  0  0  1  0  0  0  0 28  0  0]
 [ 0  1  0  1  1  0  0  0  0  0  0 29  0]
 [ 0  0  0  0  1  0  0  0  0  1  0  0 31]]


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}


100%|██████████| 413/413 [00:00<00:00, 59004.28it/s]

[[22  0  1  1  0  5  2  0  1  0  0  0  0]
 [ 0 27  0  0  4  0  0  0  0  0  0  1  1]
 [ 0  0 28  0  0  1  0  0  0  0  0  2  0]
 [ 0  1  0 22  1  0  1  3  0  0  0  2  1]
 [ 0  2  1  0 28  0  0  0  0  2  0  0  0]
 [ 4  0  0  2  0 19  6  0  0  1  0  0  0]
 [ 4  0  1  1  0  2 23  1  0  0  0  0  0]
 [ 0  0  1  4  0  1  2 24  0  0  0  0  0]
 [ 1  0  0  0  0  1  1  0 28  0  0  0  0]
 [ 0  3  0  0  0  0  0  0  0 28  0  0  1]
 [ 1  1  0  0  0  1  0  0  0  0 26  0  0]
 [ 0  1  1  2  0  0  0  0  0  0  0 28  0]
 [ 0  1  0  0  1  0  0  0  0  0  0  0 31]]


KeyboardInterrupt: 